In [29]:
import sys
from os import path
from os.path import exists
import string
from collections import Counter
from tokenize import maybe
import nltk
import matplotlib.pyplot as plt
from pathlib import Path
import nltk.corpus
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import time
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.data import load
import sagemaker
import boto3
import pandas as pd
import json

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
role = sagemaker.get_execution_role()

s3 = boto3.resource('s3')

bucket_name = 'ad440-mpg-floop-export-storage'

obj = s3.Object('ad440-mpg-floop-export-storage',
                    'auto-floop-s3-export3-sagemaker.json')

s3_client = boto3.client("s3")


# Loads body of json from the s3 object
dataset = s3_client.get_object(Bucket = 'ad440-mpg-floop-export-storage', Key = 'auto-floop-s3-export3-sagemaker.json')['Body'].read()
data = pd.read_json(dataset)

# Loads body of json from the s3 object
data = json.load(obj.get()['Body'])
print(data[0])

[{'Text': 'Eleborate and Write conclusion   3marks', 'uid': 'T'}]


In [4]:
global finalList
finalList=[]

In [8]:
dataList=[]
for x in range(10):
    dataList.append(data[x])

In [9]:
print(dataList)

[[{'Text': 'Eleborate and Write conclusion   3marks', 'uid': 'T'}], [{'Text': 'If the temperature in a pond is lower, then the dissolved oxygen will be higher because gas escapes more readily in liquids at high temperature due to increased movement of kinetic energy (= increased movement of molecules)', 'uid': 'T'}], [{'Text': 'Maybe there is a need to be clear about why they need large sums = e.g. think about the cost of the state of the art machinery that {{NAME}} uses in his factories.', 'uid': 'T'}], [{'Text': 'More details.  A lot happened.', 'uid': 'T'}, {'Text': 'iu98fgxv bm,÷\n"', 'uid': 'S'}], [{'Text': 's-v', 'uid': 'T'}], [{'Text': 'pronoun problem', 'uid': 'T'}], [{'Text': '{{NAME}}, your handwriting is hard to read...please try printing...', 'uid': 'T'}], [{'Text': 'What is this word?', 'uid': 'T'}, {'Text': 'Procedure', 'uid': 'S'}], [{'Text': '{{ORGANIZATION}}', 'uid': 'T'}], [{'Text': 'Nice paragraph!', 'uid': 'T'}]]


In [19]:
sentence=[]
for x in dataList:
    for y in x:       
        sentence.append(y['Text'])
print(sentence)

['Eleborate and Write conclusion   3marks', 'If the temperature in a pond is lower, then the dissolved oxygen will be higher because gas escapes more readily in liquids at high temperature due to increased movement of kinetic energy (= increased movement of molecules)', 'Maybe there is a need to be clear about why they need large sums = e.g. think about the cost of the state of the art machinery that {{NAME}} uses in his factories.', 'More details.  A lot happened.', 'iu98fgxv bm,÷\n"', 's-v', 'pronoun problem', '{{NAME}}, your handwriting is hard to read...please try printing...', 'What is this word?', 'Procedure', '{{ORGANIZATION}}', 'Nice paragraph!']


In [35]:
def emotion_analysis(post):
    post = post.lower()

    cleanedText = post.translate(str.maketrans('', '', string.punctuation))

    tokenizedList = word_tokenize(cleanedText, "english")
    # creating a list of words in the feedback excluding all the stop words
    finalList = [
        item for item in tokenizedList if item not in stopwords.words('english')]
    
    # creating a list of emotion words and the words that have emotion according to our pre defined list
    emotionList = []
    
    # open emotions.txt, clean the text lines, and comparegenerated finalList words with emotion words
    with open('emotions.txt', 'r') as file:
        for line in file:
            clearLine = line.replace('\n', '').replace(
                ',', '').replace("'", '').replace(' ', '').strip()
            word, emotion = clearLine.split(':')

            if word in finalList:
                emotionList.append(emotion)
                
            # returns a determined emotion from the emotionList or returns a default message
        return emotionList[0] if len(emotionList) != 0 else 'can\'t determine emotion'

In [36]:
post = "bad job"

res = emotion_analysis(post)
print(res)

sad


In [39]:
emotion_analysis(str(sentence))

'happy'